# Projeto 1 - Análise de Sentimento com Transformers

Este script em Python tem como objetivo demonstrar uma análise de sentimento com o uso de inteligência artificial, no caso com um modelo transformer. A análise de sentimento é uma ferramenta capaz de identificar se determinada mensagem, como um tweet ou post no Facebook, são positivas ou negativas. Ou seja, com Inteligência Artificial é possível identificar se uma campanha de marketing surtiu o efeito previsto com base na análise das mensagens públicas nas redes sociais sobre o tema.

## 1. Instalando os pacotes

In [1]:
# Versão da linguagem Python
from platform import python_version
print("A versão da linguagem Python usada neste Jupyter Notebook é: ", python_version())

A versão da linguagem Python usada neste Jupyter Notebook é:  3.9.13


In [3]:
# Instala o Pytorch
!pip install -q torch==2.0.0

In [4]:
# Instala o transformes
!pip install -q transformers==4.28.1

In [5]:
# Carga de Pacotes

# CSV para importar base de dados
import csv

# Pytorch
import torch
from torch.utils.data import DataLoader, Dataset

# Transformers
from transformers import BertTokenizer, BertForSequenceClassification

# Scikit-learn
from sklearn.model_selection import train_test_split

In [6]:
# Remove os avisos do Transformers
from transformers import logging
logging.set_verbosity_error()

In [7]:
# Versões dos pacotes usados neste Jupyter Notebook
%reload_ext watermark
%watermark -a "Projeto 1 - Análise de Sentimento com I.A" --iversions

Author: Projeto 1 - Análise de Sentimento com I.A

torch       : 2.0.0
csv         : 1.0
transformers: 4.28.1



## 2. Carga dos dados

In [8]:
# Caminho do arquivo CSV
csv_file_path = 'dados-projeto1/frases.csv'

In [9]:
# Cria lista para armazenar as frases
frases = []

In [10]:
# Abre o arquivo CSV no modo de leitura ('r')
# Usa o objeto 'csv.reader' para ler o arquivo
with open(csv_file_path, 'r', encoding='utf-8') as file:
    
    csv_reader = csv.reader(file)
    
    # Iterar sobre cada linha no arquivo CSV
    for row in csv_reader:
        frase = row[0]
        frases.append(frase)

In [11]:
# Visualiza as frases
print(frases)

['Eu amei este filme!', 'Este produto é terrível.', 'A comida estava deliciosa.', 'O serviço foi péssimo.', 'Este livro é incrível.', 'A história da Inglaterra é curiosa.', 'Não gostei daquele novo modelo de carro.', 'Aqueles foi um dos melhores jogos de futebol da história.', 'Este curso é fantástico e de alto nível.', 'O serviço do garçom não esteve à altura do nome do restaurante.']


In [12]:
# Nome do objeto
texts = frases

In [13]:
# Carrega os labels
# 1: positivo, 0: negativo
labels = [1, 0, 1, 0, 1, 1, 0, 1, 1, 0]

## 3. Pré-Processamento dos dados

In [14]:
RANDOM_SEED = 42

In [15]:
# Divisão dos dados em treino e teste
train_texts, test_texts, train_labels, test_labels = train_test_split(texts,
                                                                      labels,
                                                                      test_size = 0.2,
                                                                      random_state = RANDOM_SEED)


In [16]:
# Carrega o tokenizador
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [17]:
# Classe de tokenização dos dados
class SentimentAnalysisTokenizer(Dataset):
    
    # Método construtor com os parâmetros que serão determinados no treino e teste
    def __init__(self, texts, labels, tokenizer, max_length): # Método para iniciar a classe
        self.texts = texts # Extrai as frases
        self.labels = labels # Extrai os labels
        self.tokenizer = tokenizer # Registra o tokenizer
        self.max_length = max_length # Registra o comprimento máximo do texto

    # Método para calcular o comprimento do texto em cada sentença    
    def __len__(self):
        return len(self.texts) # Registra o tamanho do texto

    # Método para obter um item tokenizado
    def __getitem__(self, idx):
        
        # Obtém o indíce do texto e label
        text = self.texts[idx] # registra a frase pelo index
        label = self.labels[idx] # registra o label pelo index, mesmo da frase
        
        # Aplica a tokenização para converter a frase em matriz
        inputs = self.tokenizer.encode_plus(text, #aplica a tokenização na frase do index em text
                                            add_special_tokens = True, # indica o início e final da frase
                                            max_length = self.max_length, # comprimento máximo do texto
                                            padding = 'max_length', # Determina que as frases tem o mesmo tamanho (trunca a frase)
                                            truncation = True, # trunca o texto de acordo com o max lenght
                                            return_tensors = 'pt') # retorna tensores do tipo pytorch

        return {
            'input_ids': inputs['input_ids'].squeeze(0), # retorna a entrada da matriz
            'attention_mask': inputs['attention_mask'].squeeze(0), # retorna a attention mask
            'label': torch.tensor(label) # retorna o label
        }

In [18]:
# Comprimento máximo do texto
MAX_LENGTH = 64

In [19]:
# Aplica a tokenização
train_dataset = SentimentAnalysisTokenizer(train_texts, train_labels, tokenizer, MAX_LENGTH)

In [20]:
# Aplica a tokenização
test_dataset = SentimentAnalysisTokenizer(test_texts, test_labels, tokenizer, MAX_LENGTH)

In [21]:
# Cria o data loader
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle = True)

In [22]:
# Cria o data loader
test_loader = DataLoader(test_dataset, batch_size = 16)

## 4. Treino e Teste do Modelo

In [23]:
# Função para treinar o modelo
# criterion é a função de erro
def train_epoch(model, data_loader, criterion, optimizer, device):
    
    # Inicia o modelo em modo de treino
    # Inicia a perda do modelo
    model.train()
    total_loss = 0

    # Loop for para o batch do data loader
    for batch in data_loader:
        
        input_ids = batch['input_ids'].to(device) # extrai a entrada do modelo
        attention_mask = batch['attention_mask'].to(device) # extrai a attention mask
        labels = batch['label'].to(device) # extrai o label

        optimizer.zero_grad() # zera os gradientes
        outputs = model(input_ids, attention_mask = attention_mask, labels = labels) # treino do modelo
        loss = outputs.loss # cálcula do erro do modelo
        loss.backward() # aplica o backpropagation (otimização dos pesos para a próxima passada)
        optimizer.step()

        total_loss += loss.item() # totaliza o erro

    return total_loss / len(data_loader) # retorna o erro %

In [24]:
# Função para avaliar o modelo
# optimizer só entra no treino do modelo
def eval_epoch(model, data_loader, criterion, device):
    
    # Inicia o modelo em modo de treino
    # Inicia a perda do modelo
    model.eval()
    total_loss = 0

    # Nãos será feito o track do modelo
    with torch.no_grad():
        # Loop for para o batch do data loader
        for batch in data_loader:
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

    return total_loss / len(data_loader)

In [25]:
# Função para obter previsões do modelo
def predict(model, data_loader, device):
    
    model.eval()
    predictions = [] # cria lista com as previsões

    with torch.no_grad():
        
        for batch in data_loader:
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1) # extrai a previsão com o maior valor de logit
            predictions.extend(preds.tolist()) # inclui as previsões na lista

    return predictions

## 5. Construção do Modelo

In [26]:
# Inicialização do modelo e configuração do dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [27]:
# Carrega o modelo
# labels igual a 2 porque temos labels de 1 e 0 apenas
modelo = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)

In [28]:
# Coloca o modelo na memória do device
modelo.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [29]:
# Hiperparâmetros
EPOCHS = 10
LEARNING_RATE = 2e-5

In [30]:
# Otimizador
optimizer = torch.optim.AdamW(modelo.parameters(), lr = LEARNING_RATE)

In [31]:
# Função de perda
criterion = torch.nn.CrossEntropyLoss()

## 6. Treinamento e Avaliação do Modelo

In [34]:
# Treinamento e validação do modelo
for epoch in range(EPOCHS):
    train_loss = train_epoch(modelo, train_loader, criterion, optimizer, device)
    test_loss = eval_epoch(modelo, test_loader, criterion, device)
    print(f'Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss}, Test Loss: {test_loss}')

Epoch 1/10, Train Loss: 0.7125353813171387, Test Loss: 0.698032796382904
Epoch 2/10, Train Loss: 0.6585090160369873, Test Loss: 0.681705117225647
Epoch 3/10, Train Loss: 0.6205040216445923, Test Loss: 0.7412533164024353
Epoch 4/10, Train Loss: 0.6916980743408203, Test Loss: 0.7378116250038147
Epoch 5/10, Train Loss: 0.5738108158111572, Test Loss: 0.7376834154129028
Epoch 6/10, Train Loss: 0.5448307991027832, Test Loss: 0.7551488280296326
Epoch 7/10, Train Loss: 0.48689982295036316, Test Loss: 0.7095967531204224
Epoch 8/10, Train Loss: 0.46572527289390564, Test Loss: 0.7322322130203247
Epoch 9/10, Train Loss: 0.47990214824676514, Test Loss: 0.73696368932724
Epoch 10/10, Train Loss: 0.4394643008708954, Test Loss: 0.7230180501937866


In [35]:
# Salva o modelo em disco
torch.save(modelo, 'modelos/modelo_dsa_mp1.pt')

In [36]:
# Carrega o modelo do disco
modelo_final = torch.load('modelos/modelo_dsa_mp1.pt')

## 7. Testando o Modelo com Novos Dados

In [37]:
# Novos dados
novas_frases = ['Eu gostei muito deste filme.',
                'O atendimento do restaurante foi decepcionante.']

In [38]:
# Aplica a tokenização
dataset = SentimentAnalysisTokenizer(novas_frases, [0] * len(novas_frases), tokenizer, MAX_LENGTH)

In [39]:
# Cria o data loader
loader = DataLoader(dataset, batch_size = 16)

In [40]:
# Previsões
previsoes = predict(modelo_final, loader, device)

In [41]:
# Análise de sentimento
for text, prediction in zip(novas_frases, previsoes):
    print(f'Sentença: {text} | Sentimento: {"positivo" if prediction else "negativo"}')

Sentença: Eu gostei muito deste filme. | Sentimento: negativo
Sentença: O atendimento do restaurante foi decepcionante. | Sentimento: negativo
